# TextAttack on Keras Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_3_Keras.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_3_Keras.ipynb)

## Training

The code below trains a basic neural network on a series of movie reviews from the IMDB dataset, loaded using Tensorflow's datasets module. Each review is encoded as a sequence of tokens corresponding to a word's index in the vocabulary. Class labels are provided, denoting a positive or negative sentiment. 

See [here](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data) for more information on the IMDB dataset. 


In [ ]:
import tensorflow as tf
import keras
import os
import numpy as np
from keras.utils import to_categorical
import torch
import textattack
from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019


import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:16<00:00, 29.8MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpdmzgnt8b.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


Below, we load the IMDB dataset from Tensorflow and transform it for our classifier, using a Bag-of-Words format. 

In [ ]:

NUM_WORDS = 10000

(x_train_tokens, y_train), (x_test_tokens, y_test) = tf.keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=NUM_WORDS,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)


def transform(x):
  x_transform = []
  for i, word_indices in enumerate(x):
    BoW_array = np.zeros((NUM_WORDS,))
    for index in word_indices:
      if index < len(BoW_array):
        BoW_array[index] += 1
    x_transform.append(BoW_array)
  return np.array(x_transform)
    

index = int(0.9 * len(x_train_tokens))
x_train = transform(x_train_tokens)[:index]
x_test = transform(x_test_tokens)[index:]
y_train = np.array(y_train[:index])
y_test = np.array(y_test[index:])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

17465344/17464789 [==============================] - 0s 0us/step
(22500, 10000) (22500, 2) (2500, 10000) (2500, 2)


With our data successfully loaded, we can now design and trained our model. 

In [ ]:

#Model Created with Keras
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=NUM_WORDS))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))



opt = keras.optimizers.Adam(learning_rate=0.00001)

model.compile(
 optimizer = opt,
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)


results = model.fit(
 x_train, y_train,
 epochs= 18,
 batch_size = 512,
 validation_data = (x_test, y_test)
)


print(results.history)


<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fa0458fee10>>
Epoch 1/18
44/44 [==============================] - 3s 28ms/step - loss: 0.7069 - accuracy: 0.5209 - val_loss: 0.6731 - val_accuracy: 0.6240
Epoch 2/18
44/44 [==============================] - 1s 16ms/step - loss: 0.6760 - accuracy: 0.6041 - val_loss: 0.6543 - val_accuracy: 0.6840
Epoch 3/18
44/44 [==============================] - 1s 16ms/step - loss: 0.6597 - accuracy: 0.6453 - val_loss: 0.6353 - val_accuracy: 0.7192
Epoch 4/18
44/44 [==============================] - 1s 16ms/step - loss: 0.6378 - accuracy: 0.6942 - val_loss: 0.6152 - val_accuracy: 0.7464
Epoch 5/18
44/44 [==============================] - 1s 16ms/step - loss: 0.6181 - accuracy: 0.7217 - val_loss: 0.5942 - val_accuracy: 0.7808
Epoch 6/18
44/44 [==============================] - 1s 24ms/step - loss: 0.5933 - accuracy: 0.7553 - val_loss: 0.5729 - val_accuracy: 0.8008
Epoch 7/18
44/44 [========================

## Attacking

With our model trained, we can create a  `ModelWrapper` that will allow us to run TextAttack on a custom Keras model. Each `ModelWrapper` must implement a single method, `__call__`, which takes a list of strings and returns a `List`, `np.ndarray`, or `torch.Tensor` of predictions.

In [ ]:


class CustomKerasModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
      text_array = np.array([words2tokens(text_input) for text_input in text_input_list])
      prediction = self.model.predict(text_array)
      preds = [list(prediction[i][0]) for i in range(0, len(prediction))]
      return preds


CustomKerasModelWrapper(model)(["the movie was awful", "the movie was awesome"])


[[0.55557764, 0.44437635], [0.4965529, 0.50741225]]

With our `ModelWrapper` constructed, we can use TextAttack's HuggingFaceDataset module to load reviews for testing, alongside TextAttack's PWWSRen2019 module to serve as our attack recipe. 

The attack below leverages TextAttack's `Attack` class, capable of running attacks against entire datasets. 


In [ ]:
model_wrapper = CustomKerasModelWrapper(model)
dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)

attack = PWWSRen2019.build(model_wrapper)

results_iterable = attack.attack_dataset(dataset, indices=range(10))
for result in results_iterable:
  print()
  print()
  print(result.__str__(color_method='ansi'))

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a)
textattack: Loading datasets dataset rotten_tomatoes, split test.
textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.




Positive (50%) --> [SKIPPED]

movies like high crimes flog the dead horse of surprise as if it were an obligation . how about surprising us by trying something new ?


Positive (53%) --> [SKIPPED]

in a 102-minute film , aaliyah gets at most 20 minutes of screen time . . . . most viewers will wish there had been more of the " queen " and less of the " damned . "


Negative (51%) --> Positive (50%)

more likely to have you scratching your head than hiding under your seat .

more probably to have you cancel your psyche than hiding under your seat .


Negative (50%) --> Positive (51%)

slow , silly and unintentionally hilarious .

easy , silly and unintentionally hilarious .


Positive (50%) --> Negative (52%)

by its modest , straight-ahead standards , undisputed scores a direct hit .

by its modest , straight-ahead standards , undisputed scores a direct shoot .


Positive (51%) --> [SKIPPED]

novak contemplates a heartland so overwhelmed by its lack of purpose that it seeks excitement

## Conclusion

Great! We trained a binary classifier, created a custom `ModelWrapper` for Keras models, and successsfully ran adversarial attacks against our trained Keras model! This serves a basic demo for how to use TextAttack within your own environments. 
